## Creación de modelos para el analísis de imagenes de 5 tipos de insectos para su clasificación

**Importación de librerias**

In [2]:
import os
import random
import shutil
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from PIL import Image
import numpy as np

In [3]:
# Ruta de la carpeta principal del dataset
dataset_path = "../Faces"

# Ruta de la carpeta de entrenamiento y prueba
train_path = "../FacesDataset/train"
test_path = "../FacesDataset/test"

# Proporción de división (80% para entrenamiento, 20% para prueba)
train_ratio = 0.8

In [4]:
#Parámetros para el aumento de imagenes
datagen = ImageDataGenerator(
    horizontal_flip=True,
    vertical_flip=True,
    rescale=1. / 255,
    shear_range=0.1,
    zoom_range=0.1,
    width_shift_range=0.1,
    height_shift_range=0.1,
    rotation_range=30)


In [5]:
for class_folder in os.listdir(dataset_path):
    class_path = os.path.join(dataset_path, class_folder)

    # Verifica si el elemento en la ruta es una carpeta
    if os.path.isdir(class_path):
        # Crea las carpetas correspondientes en el conjunto de entrenamiento y prueba
        train_class_folder = os.path.join(train_path, class_folder)
        test_class_folder = os.path.join(test_path, class_folder)
        os.makedirs(train_class_folder, exist_ok=True)
        os.makedirs(test_class_folder, exist_ok=True)

        # Obtiene la lista de archivos de imágenes en la carpeta de clase
        images = os.listdir(class_path)

        # Baraja aleatoriamente las imágenes
        random.shuffle(images)

        # Calcula el índice para dividir las imágenes
        train_size = int(len(images) * train_ratio)

        # Divide las imágenes en las sublistas de entrenamiento y prueba
        train_images = images[:train_size]
        test_images = images[train_size:]

        # Genera las imágenes aumentadas y las guarda en la carpeta de entrenamiento
        for image in train_images:
            src_path = os.path.join(class_path, image)
            dst_path = os.path.join(train_class_folder, image)

            # Carga la imagen utilizando PIL
            img = Image.open(src_path)

            # Convierte la imagen a un array numpy
            img_array = np.array(img)

            # Reshape del array para que tenga dimensiones (1, height, width, channels)
            img_array = img_array.reshape((1,) + img_array.shape)

            # Genera imágenes aumentadas utilizando ImageDataGenerator
            augmented_images = datagen.flow(
                img_array, batch_size=1, save_to_dir=train_class_folder, save_prefix="aug_", save_format="png")

            # Guarda las imágenes aumentadas generadas
            for i, augmented_image in enumerate(augmented_images):
                if i >= len(augmented_images):
                    break

        for image in test_images:
            src_path = os.path.join(class_path, image)
            dst_path = os.path.join(test_class_folder, image)
            shutil.copy(src_path, dst_path)
